In [ ]:
! pip install openpyxl

In [1]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import pandas as pd

In [2]:
driver = webdriver.Firefox()


In [3]:
login_page_url = "https://keycloak.atingi.org/realms/master/protocol/openid-connect/auth?client_id=moodle&response_type=code&redirect_uri=https%3A%2F%2Fonline.atingi.org%2Fadmin%2Foauth2callback.php&state=%2Fauth%2Foauth2%2Flogin.php%3Fwantsurl%3Dhttps%253A%252F%252Fonline.atingi.org%252Fcourse%252Fview.php%253Fid%253D1161%26sesskey%3DEXX7VnZ02Q%26id%3D1&scope=openid%20profile%20email&ui_locales=en"

In [4]:
driver.get(login_page_url)
time.sleep(2)


In [5]:
email_field = driver.find_element("id","username")
password_field = driver.find_element("id","password")
login_button = driver.find_element("id","kc-login")
email_field.send_keys("<email>")
time.sleep(2)
password_field.send_keys("<password>")
time.sleep(2)
login_button.click()

In [6]:
url = "https://online.atingi.org/local/explore/"

In [7]:
driver.get(url)

In [8]:
language_container = driver.find_element("xpath","//div[@class='mb-4']")
languages = language_container.find_elements("xpath","//div[@class='text_to_html']")
for language in languages:
    if language.text == "English":
        language.click()
        
# time.sleep(2)
pages = driver.find_elements("xpath","//a[@class='page-link']")
for page in pages:
    # print("page text: ",page.text)
    if "Show All" in page.text:
        page.click()

In [9]:
course_links = [course.get_attribute("href") for course in driver.find_elements("xpath","//a[@class='btn btn-primary m-t-1']")]


In [11]:
course_names = [course_name.text.strip() for course_name in driver.find_elements("xpath","//span[@class='multiline coursename']")]

In [19]:
for course_link,course_name in zip(course_links,course_names):
# for course_link,course_name in zip(python,python_name):
    main_window_handle = driver.current_window_handle
    files = os.listdir("courses")
    if course_name+".txt" in files:
        print("Skipping bcs the file is already present, {}..........".format(course_name))
        continue
    if course_name in selected_courses:
        print("Skipping course: ",course_name," because its  in the selected courses.")
        continue
    # if course_name not in selected_courses:
    driver.get(course_link)
    driver.maximize_window()
    print("Course name: ",course_name)
    try:
        enrol_button = driver.find_element("xpath",'//input[@value="Enrol me"]')
        enrol_button.click()
    except Exception as e:
        # print(e)
        print("You are either already enrolled or the course is off-limit")

    course_contents = driver.find_elements("xpath","//a[@class='tile-link']")
    if len(course_contents) == 0:
        course_contents = driver.find_elements("xpath","//a[@class='aalink']")
    course_content_links = [link.get_attribute("href") for link in course_contents]
    print("course links: ",course_content_links)
    if len(driver.window_handles) > 1:
        handles = driver.window_handles

        # Switch to the second window (index 1)
        driver.switch_to.window(handles[1])

        # Close the second window
        driver.close()
        driver.switch_to.window(main_window_handle)
    for lesson_link in course_content_links:
        #get text then 
        print("link: ",lesson_link)
        try:
            driver.get(lesson_link)
        except Exception as e:
            print("course content link failed: ",lesson_link)
            continue
        time.sleep(2)
        # is_pop_up = 0
        activity_flag = 0
        enter_flag = 0
        # elements = driver.find_elements("xpath","//span")
        # for element in elements:
            # print(element.text)
        print(".........................")
        try:
            activity = driver.find_element("xpath","//span[@class='activitytitle']").click()
            print("Just clicked on the activity...............")
            # time.sleep(9)
            # if len(driver.window_handles) > 1:
                # is_pop_up = 1
                # window_handler = driver.window_handles[1]
                # driver.switch_to.window(window_handler)
        except Exception as e:
            # print(e)
            print("Activity not detected..............")
            flag = input("Y/n: ")
            if flag == "n":
                continue
            if flag == "s":
                break
            activity_flag = 1
        time.sleep(10)
        try:
            enter_button = driver.find_element("xpath",'//input[@value="Enter"]').click()
            print("Clicked on the enter button......................")
            time.sleep(3)

        except Exception as e:
            # print(e)
            print("Enter button not detected..........")
            print("*******************************")
            enter_flag = 1
            flag = input("Y/n: ")
            if flag == "n":
                continue
            if flag == "s":
                break
        if len(driver.window_handles) > 1:
                # is_pop_up = 1
            window_handler = driver.window_handles[-1]
            driver.switch_to.window(window_handler)
        
        # if enter_flag == 1 and activity_flag == 1:
            # continue
        in_frame = 0
        max_continues = 16
        iterations = 0
        clicked = 0
        
        while True:
            iterations += 1
            if iterations > max_continues:
                break
            print("Sanity check level 1..............")
            if driver.current_window_handle != main_window_handle:
                WebDriverWait(driver, 240).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
                print("Successfully switched to pop-up window!")

            elif driver.current_window_handle == main_window_handle:
                WebDriverWait(driver, 240).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
                print("Failed to switch to pop-up window.")
            try:
                #find the continue button
                # time.sleep(2)
 

                # if in_frame == 0:
                
                # 
                    # in_frame = 1
      
                # in_frame = 0
                try:
                    time.sleep(5)
                    iframe = driver.find_element("css selector","iframe")
                    # wait = WebDriverWait(driver, 60)
                    # frame = wait.until(EC.frame_to_be_available_and_switch_to_it("css selector","iframe"))
                    driver.switch_to.frame(iframe)
                    print("Switched frame........")
                except Exception as e:
                    print("Couldn't find the frame.........")
                    break
                print("After the frame process........")
                # time.sleep(5)
                
                
                is_alt_start = 1
                try:
                    if clicked == 0:
                        start_class_button = driver.find_elements("xpath","//span")
                        for element in start_class_button:
                            if "START COURSE" in element.text or "START" in element.text:
                                is_alt_start = 0
                                clicked = 1
                                element.click()
                                time.sleep(3)
                        if is_alt_start == 1:
                            alt_start_button = driver.find_elements("xpath","//a")
                            for element in alt_start_button:
                                if "START" in element.text:
                                    element.click()
                                    clicked = 1
                                    time.sleep(3)
                except Exception as e:
                    print("No start button...........")
                try:
                    menu = driver.find_element("xpath","//button[@aria-label='Open navigation menu']").click()
                except Exception as e:
                    print("no navigation menu found.")
                try: 
                    # old_topic_links = driver.find_elements("css selector","a[data-link='lesson-link-item']")
                    # list_of_links = driver.find_element("xpath","//li[@class='lesson-lists__item']")
                    try:
                        list_of_links = driver.find_element("xpath","//ol[@class='lesson-lists__list']")
                    except Exception as e:
                    # if len(list_of_links) == 0:
                        list_of_links = driver.find_element("xpath","//ol[@class='nav-sidebar__outline-list']")
                    print("list of links: ",list_of_links)
                    old_topic_links = list_of_links.find_elements("css selector","a")
                    if len(old_topic_links) == 0:
                        old_topic_links = driver.find_elements("xpath","//a[@class='lesson-link']")
                        
                    print("old_topics-: ",old_topic_links)
                    topic_links = []
                    [topic_links.append(element) for element in old_topic_links if element not in topic_links]
                    print("##############################33")
                    number_of_topics = len(topic_links)
                    print("Number of topic links: ",number_of_topics)
                except Exception as e:
                    print("Failed to fetch lesson links..........")
                    continue

                
                for topic_index,link in enumerate(topic_links):
                    print("link text: ",link.text)
                    try:
                        link.click()
                        time.sleep(2)
                    except Exception as e:
                        print("Link not clickable...............")
                    try:
                        while True:
                            print("We are here................")
                            if_continue = 0
                            paragraphs = driver.find_elements("xpath","//p")
                            # print("paragraphs: ",paragraphs)
                            with open(str(course_name)+".txt","a") as f:
                                for paragraph in paragraphs:
                                    # print("paragraph: ",paragraph.text)
                                    line = paragraph.get_attribute("textContent")
                                    print("text: ",line)
                                    f.write(line+"\n")
                            # if clicked == 0:
                            try:
                                scrollable_element = driver.find_element("id","page-wrap")
                            except Exception as e:
                                print("Scrollable element not found .............")
                            # scroll_amount = 1000
                            # driver.execute_script("arguments[0].scrollBy(0, arguments[1]);", scrollable_element, scroll_amount)
                            start_time = time.time()
                            time_out = 40
                            print("before scrolling +++++++++++++++++++++++++")
                            try:
                                while True:
                                    print("Get the current height of the scrollable element ..............")
                                    current_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
                                    # Scroll to the bottom of the scrollable element
                                    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_element)
                                    # Wait for the scrollable element to load new content
                                    time.sleep(1)
                                    print("Get the new height of the scrollable element...................")
                                    new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
                                    # If the new height is the same as the current height, we have reached the end of the scrollable element
                                    if new_height == current_height:
                                        break
                                    elapsed_time = time.time() - start_time
                                    print("Elapsed_time: ",elapsed_town)
                                    print("Time out: ",time_out)
                                    print("***************************")
                                    if elapsed_time > time_out:
                                        break
                            except Exception as e:
                                print("scrolling failed ###############")
                            time.sleep(2)
                            try: 
                                continue_buttons = driver.find_elements("css selector","button")#[@class='continue-btn brand--ui brand--background']")
                                for button in continue_buttons:
                                    print("button.text: ",button.text)
                                    if "CONTINUE" in button.text or "Continue" in button.text or "continue" in button.text or "LET" in button.text or "Let" in button.text or "let" in button.text or "Click" in button.text or "click" in button.text:
                                        print("Used the continue button..............")
                                        button.click()
                                        if_continue = 1
                                        time.sleep(2)
                                        # in_frame = 1
                                        continue
                                if if_continue == 0:                            
                                    break
                            except Exception as e:
                                print("Pressing the continue button failed ==============")
                    except Exception as e:
                        print("Failed while getting text..............")




            except Exception as e:
                print("Something went wrong :( ...............")
                print(e)
                print("\n****************************************\n")
                # if is_pop_up == 1:
                if len(driver.window_handles) > 1:
                    # driver.close()
                    driver.switch_to.window(main_window_handle)
                # go to the next lesson linkclass="continue-btn brand--ui brand--background"
                break
        driver.switch_to.default_content()
        if len(driver.window_handles) > 1:
                    # driver.close()
            driver.switch_to.window(main_window_handle)


Course name:  Microlearning nugget: Use a search engine
You are either already enrolled or the course is off-limit
course links:  ['https://online.atingi.org/course/view.php?id=2104&section=1', 'https://online.atingi.org/course/view.php?id=2104&section=2', 'https://online.atingi.org/course/view.php?id=2104&section=3']
link:  https://online.atingi.org/course/view.php?id=2104&section=1
.........................
Just clicked on the activity...............
Enter button not detected..........
*******************************


Y/n:  y


Sanity check level 1..............
Failed to switch to pop-up window.
Switched frame........
After the frame process........
no navigation menu found.
Failed to fetch lesson links..........
Sanity check level 1..............
Failed to switch to pop-up window.
Switched frame........
After the frame process........
no navigation menu found.
list of links:  <selenium.webdriver.remote.webelement.WebElement (session="bf900020-c035-46cc-8e0a-4b85575ce434", element="b1e5cf1d-3b56-4d91-96b6-35bf41d0c696")>
old_topics-:  [<selenium.webdriver.remote.webelement.WebElement (session="bf900020-c035-46cc-8e0a-4b85575ce434", element="c6eaff63-e740-43f9-9ddc-cc081298e7fe")>, <selenium.webdriver.remote.webelement.WebElement (session="bf900020-c035-46cc-8e0a-4b85575ce434", element="2b6cfd00-a612-4e08-84cd-3aa6905c4a16")>, <selenium.webdriver.remote.webelement.WebElement (session="bf900020-c035-46cc-8e0a-4b85575ce434", element="98351e20-221d-4c9c-a374-d0b873d02018")>, <selenium.webdriver.remote.webelement

Y/n:  s


Course name:  Tour Guiding 3: Key Guiding Skills II - How to talk about...
You are either already enrolled or the course is off-limit
course links:  ['https://online.atingi.org/course/view.php?id=2466&section=1', 'https://online.atingi.org/course/view.php?id=2466&section=2', 'https://online.atingi.org/course/view.php?id=2466&section=3', 'https://online.atingi.org/course/view.php?id=2466&section=4']
link:  https://online.atingi.org/course/view.php?id=2466&section=1
.........................
Just clicked on the activity...............
Clicked on the enter button......................
Sanity check level 1..............
Successfully switched to pop-up window!
Switched frame........
After the frame process........
no navigation menu found.
Failed to fetch lesson links..........
Sanity check level 1..............
Successfully switched to pop-up window!
Switched frame........
After the frame process........
No start button...........
no navigation menu found.
list of links:  <selenium.webdriv

...

Y/n:  s


Course name:  AI & Nigeria: Exploring Threats to Human Rights
course links:  ['https://online.atingi.org/course/view.php?id=3036&section=1', 'https://online.atingi.org/course/view.php?id=3036&section=2', 'https://online.atingi.org/course/view.php?id=3036&section=3']
link:  https://online.atingi.org/course/view.php?id=3036&section=1
.........................
Activity not detected..............


Y/n:  s


Course name:  The role of culture in AI: An Eastern perspective
course links:  ['https://online.atingi.org/course/view.php?id=3175&section=1', 'https://online.atingi.org/course/view.php?id=3175&section=2', 'https://online.atingi.org/course/view.php?id=3175&section=3']
link:  https://online.atingi.org/course/view.php?id=3175&section=1
.........................
Activity not detected..............


Y/n:  s
